In [1]:
!pip install tweepy

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install python-dotenv

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [3]:
import tweepy
import pandas as pd
import dotenv
import os

In [4]:
dotenv.load_dotenv()
os.environ['BEARER_TOKEN']

'AAAAAAAAAAAAAAAAAAAAAGvpiwEAAAAAl8MabBoSfHpYXr26GqLb0x5JT5Q%3Du47KzCwQzI9xrJq212elyiIEMi5rhwvvqZfi8CbSfIHieOs53z'

In [5]:
client = tweepy.Client(os.environ['BEARER_TOKEN'])

In [19]:
user = client.get_user(username="sadedposts")

In [7]:
user.data.id

1270882326742999040

In [42]:
tweets = []
user_tweets = client.get_users_tweets(user.data.id,tweet_fields=["author_id","created_at","public_metrics"],exclude='retweets',max_results=100)

for i in range(1,1000):
    if("next_token" in user_tweets.meta.keys()):
        next_token= user_tweets.meta["next_token"]
        user_tweets = client.get_users_tweets(user.data.id,tweet_fields=["author_id","created_at","public_metrics"],exclude='retweets',max_results=100,pagination_token=next_token)
        tweets.append([x.text for x in user_tweets.data])


In [58]:
test_tweets = (pd.DataFrame(tweets[0],columns=["post_text"]))

In [11]:
#SPARK
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('nlp').config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.2.0").getOrCreate()


:: loading settings :: url = jar:file:/root/venv/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-18a8ef15-0451-4cf7-9dd3-f56b75332637;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.2.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-

In [12]:
from pyspark.ml.feature import StopWordsRemover
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel
from pyspark.ml.classification import NaiveBayes


In [14]:
nb_classifier = PipelineModel.load("nb_classifier")

In [59]:
from nltk.tokenize import TweetTokenizer

tk = TweetTokenizer(reduce_len=True,strip_handles=True)

tk_tokens = [tk.tokenize(tweet) for tweet in test_tweets["post_text"].tolist() if True]
tk_tweets = [" ".join(token) for token in tk_tokens if True] 

test_tweets["tk_tweet"] = tk_tweets
test_tweets


,post_text,tk_tweet
0,maybe its better to keep it inside.,maybe its better to keep it inside .
1,"i disappear sometimes, that's my thing","i disappear sometimes , that's my thing"
2,you’re the risk i’m willing to take.,you ’ re the risk i ’ m willing to take .
3,kinda wanna disappear for a while.,kinda wanna disappear for a while .
4,"i miss you, i miss us, i miss everything with you","i miss you , i miss us , i miss everything wit..."
...,...,...
95,crying silently is the worst.,crying silently is the worst .
96,"hi, i love you","hi , i love you"
97,i wanna make cute memories with you,i wanna make cute memories with you
98,i still remember the first day i saw you,i still remember the first day i saw you


In [62]:
tk_tweets = spark.createDataFrame(test_tweets)

In [65]:
nb_classifier.transform(tk_tweets).show(truncate=False)

22/11/28 03:28:52 WARN DAGScheduler: Broadcasting large task binary with size 8.2 MiB
+---------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------+---------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=05afff6f-eed1-4617-8c5f-e922b7f4b15f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>